In [6]:
# JOIN THE TWO MULTIPLEMAPPING MAVIR 3 and MAVIR 7 files
def merge_files(file1, file2, output_file):
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        content1 = f1.read()
        content2 = f2.read()

    with open(output_file, 'w') as out_file:
        out_file.write(content1)
        out_file.write(content2)


In [7]:
# create another file with just the DEV words
def onlydev_lines(input_file, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            if line.startswith('DEV'):
                f_out.write(line)


In [8]:
#now a file with unique DEV words 
def find_unique_words(input_file):
    unique_words = []
    with open(input_file, 'r') as f_in:
        for line in f_in:
            word = line.strip().split()[0]  # Extract the first word
            if word not in unique_words:
                unique_words.append(word)
    return unique_words

def filter_lines_with_unique_words(input_file, unique_words, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            word = line.strip().split()[0]  # Extract the first word
            if word in unique_words:
                f_out.write(line)
                


In [9]:
#create the XML file.

#now a file with unique DEV words 
def find_unique_words(input_file):
    unique_words = []
    with open(input_file, 'r') as f_in:
        for line in f_in:
            word = line.strip().split()[0]  # Extract the first word
            if word not in unique_words:
                unique_words.append(word)
    return unique_words

def filter_lines_with_unique_words(input_file, unique_words, output_file):
    with open(output_file, 'w') as f_out:
        f_out.write('<stdlist termlist_filename="C:\\Users\\Adriana\\Desktop\\TFG\\resultados\\onlydevmatrizswapmultiple.xml" indexing_time="1.000" language="spanish" index_size="1" system_id="fake">\n')
        for word in unique_words:
            f_out.write(f'<detected_termlist termid="{word}" term_search_time="24.3" oov_term_count="1">\n')
            with open(input_file, 'r') as f_in:
                for line in f_in:
                    if line.startswith(word):
                        columns = line.strip().split()  # Split the line into columns
                        term_file = columns[5]  # Get the term file (sixth column)
                        tbeg = columns[1]  # Get the tbeg (second column)
                        dur = columns[4]  # Get the dur (fifth column)
                        score = columns[3]  # Get the score (fourth column)
                        f_out.write(f'<term file="{term_file}" channel="1" tbeg="{tbeg}" dur="{dur}" score="{score}" decision="YES"/>\n')
            f_out.write('</detected_termlist>\n')
        f_out.write('</stdlist>\n')



In [11]:
# now that we have the xml file, we must create another one in which words are organized in ascending 
# time order
# <detected_termlist termid="DEV-0253" term_search_time="24.3" oov_term_count="1">
#<term file="mavir07" channel="1" tbeg="1.7" dur="0.5" score="0.995" decision="YES"/>
#<term file="mavir03" channel="1" tbeg="419.28" dur="0.44" score="0.994" decision="YES"/>
#<term file="mavir03" channel="1" tbeg="1111.44" dur="0.38" score="0.992" decision="YES"/>
#</detected_termlist>
# write the first line in the new output file
#write the line of the detected termlist in the new output file
# put the terms in order of increasing tbeg, with the rest of the info from the other columns
#add this created list to the outputfile.
# two steps: the first one, if it finds a word with a tbeg that is higher, replace it by that one
#check for the replaced word, if there is any word that is greater, if so replace, and do this until
#there are none.
#open the input file again and replace with whole info?
#into a list. Stop when it reaches detected_termlist.
#once you have the values, 
#wirte the line of the final detected termlist in the output file
#after every iteration, write stdlist. 

def sort_lines_within_termlist(detected_termlist):
    terms =  []
    sorted_lines = []
    sorted_numbers =[]
    for line in detected_termlist:
        parts = line.strip().split()  # Split the line into parts
        tbeg_part = parts[3]
        print(tbeg_part)
        tbeg_value = float(tbeg_part.split('=')[1].strip('"'))  # Extract the numerical value
        terms.append((tbeg_value))
    sorted_numbers = sorted(terms)
    for term in sorted_numbers:
        print("here are the sorted numbers")
        print(sorted_numbers)
    #match with the content of the detected_termlist, to properly order it, and write it into the ordered_list
    for number in sorted_numbers:
        for line in detected_termlist:
            partsofline = line.strip().split()
            tbeg_part = partsofline[3]
            tbeg_value = float(tbeg_part.split('=')[1].strip('"'))
            if number == tbeg_value:
                sorted_lines.append(line)
    return sorted_lines

def order_detected_termlists(input_file, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        first_line = f_in.readline()  # Read the first line
        f_out.write(first_line)  # Write the first line to the output file

        detected_termlist = []  # To store lines within detected_termlist block

        for line in f_in:
            if line.startswith('<detected_termlist'):
                f_out.write(line)  # Write the line to the output file if it matches the specified format
            elif line.startswith('</detected_termlist>'):
                sorted_lines = sort_lines_within_termlist(detected_termlist)
                for sorted_line in sorted_lines:
                    f_out.write(sorted_line)  # Write sorted lines to the output file
                detected_termlist = []  # Reset detected_termlist
                f_out.write(line)  # Write the line to the output file
            elif line.startswith('<term'):  # Store lines within detected_termlist block
                detected_termlist.append(line)
            elif line.startswith('</stdlist>'):
                f_out.write(line)
       